# Force Fields

A `ForceField` defines the interaction parameters needed for molecular simulations. It specifies how atoms interact through bonds, angles, dihedrals, and non-bonded forces.

**What is a Force Field?**  
A collection of potential energy functions and parameters that describe:
- **Atom types**: Mass, charge, and van der Waals parameters
- **Bond interactions**: Harmonic springs connecting atoms
- **Angle interactions**: Three-body angular potentials
- **Dihedral interactions**: Four-body torsional potentials
- **Pair interactions**: Non-bonded Coulomb and Lennard-Jones forces

**Two ways to use Force Fields:**
1. **Load & Apply**: Load a standard FF (OPLS-AA, AMBER, GAFF) and apply it to your molecule (see [Typifier Guide](../user-guide/typifier.ipynb))
2. **Define Manually**: Create custom parameters from scratch (this tutorial)

**Why it matters:**
- ✅ Simulation engines (LAMMPS, GROMACS) require force field parameters
- ✅ Correct parameters → physically realistic simulations
- ✅ Custom force fields enable novel material modeling

---

## 1. Creating a Force Field

Let's start by creating an empty force field and understanding its structure.

In [23]:
from molpy.core.forcefield import ForceField, AtomStyle, BondStyle, AngleStyle, DihedralStyle
from molpy.core.forcefield import AtomType, BondType, AngleType, DihedralType

# Initialize an empty force field
ff = ForceField(name="MyCustomFF")

print(ff)
print(f"\nForce field name: {ff.name}")
print(f"Units: {ff.units}")

<ForceField: MyCustomFF>

Force field name: MyCustomFF
Units: real


## 2. Defining Atom Types

Atom types are the foundation of a force field. Each type has:
- **Name**: Unique identifier (e.g., "CT", "HC", "O_hydroxyl")
- **Mass**: Atomic mass in g/mol
- **Charge**: Partial charge in elementary charge units
- **LJ parameters** (optional): σ (sigma) and ε (epsilon) for Lennard-Jones interactions

**Common naming conventions:**
- OPLS: CT (carbon tetrahedral), HC (hydrogen on carbon), OH (hydroxyl oxygen)
- AMBER: C (sp2 carbon), CA (aromatic carbon), N (sp2 nitrogen)

In [24]:
# Define atom style (common styles: 'atomic', 'full', 'molecular')
# 'full' includes bonds, angles, dihedrals, and charges
atom_style = ff.def_style(AtomStyle, "full")

# Use the style's def_type() method to create atom types
ct = atom_style.def_type("CT", mass=12.011, charge=-0.18)  # Aliphatic carbon
hc = atom_style.def_type("HC", mass=1.008, charge=0.06)    # Hydrogen on carbon

# Add Lennard-Jones parameters (optional)
# sigma in Angstroms, epsilon in kcal/mol
ct["sigma"] = 3.50
ct["epsilon"] = 0.066

hc["sigma"] = 2.50
hc["epsilon"] = 0.030

print(f"Defined {len(ff.get_types(AtomType))} atom types:")
for atype in ff.get_types(AtomType):
    mass = atype["mass"]
    charge = atype["charge"]
    print(f"  {atype.name}: mass={mass:.3f}, charge={charge:.3f}")

Defined 2 atom types:
  CT: mass=12.011, charge=-0.180
  HC: mass=1.008, charge=0.060


## 3. Defining Bond Parameters

Bonds are typically modeled as harmonic springs:

$$E_{bond} = k(r - r_0)^2$$

Where:
- $k$ = force constant (kcal/mol/Å²)
- $r$ = current bond length
- $r_0$ = equilibrium bond length (Å)

**Common bond styles:**
- `harmonic`: Standard quadratic potential
- `morse`: Anharmonic potential for bond breaking
- `fene`: Finitely extensible nonlinear elastic (for polymers)

In [25]:
# Define bond style
bond_style = ff.def_style(BondStyle, "harmonic")

# Use the style's def_type() method to create bond types
ct_hc_bond = bond_style.def_type(ct, hc, k=340.0, r0=1.09)   # C-H bond
ct_ct_bond = bond_style.def_type(ct, ct, k=268.0, r0=1.529)  # C-C bond

print(f"Defined {len(ff.get_types(BondType))} bond types:")
for btype in ff.get_types(BondType):
    k = btype["k"]
    r0 = btype["r0"]
    print(f"  {btype.name}: k={k:.1f} kcal/mol/Å², r0={r0:.3f} Å")

Defined 2 bond types:
  CT-CT: k=268.0 kcal/mol/Å², r0=1.529 Å
  CT-HC: k=340.0 kcal/mol/Å², r0=1.090 Å


## 4. Defining Angle Parameters

Angles describe the energy associated with bending between three bonded atoms:

$$E_{angle} = k(\theta - \theta_0)^2$$

Where:
- $k$ = force constant (kcal/mol/rad²)
- $\theta$ = current angle
- $\theta_0$ = equilibrium angle (degrees)

In [26]:
# Define angle style
angle_style = ff.def_style(AngleStyle, "harmonic")

# Use the style's def_type() method to create angle types
hc_ct_hc = angle_style.def_type(hc, ct, hc, k=33.0, theta0=107.8)   # H-C-H angle
hc_ct_ct = angle_style.def_type(hc, ct, ct, k=37.5, theta0=110.7)   # H-C-C angle
ct_ct_ct = angle_style.def_type(ct, ct, ct, k=58.35, theta0=112.7)  # C-C-C angle

print(f"Defined {len(ff.get_types(AngleType))} angle types:")
for atype in ff.get_types(AngleType):
    k = atype["k"]
    theta0 = atype["theta0"]
    print(f"  {atype.name}: k={k:.2f} kcal/mol/rad², θ0={theta0:.1f}°")

Defined 3 angle types:
  CT-CT-CT: k=58.35 kcal/mol/rad², θ0=112.7°
  HC-CT-HC: k=33.00 kcal/mol/rad², θ0=107.8°
  HC-CT-CT: k=37.50 kcal/mol/rad², θ0=110.7°


## 5. Defining Dihedral Parameters

Dihedrals (torsions) control rotation around bonds. Common forms include:

**OPLS style:**
$$E_{dihedral} = \frac{1}{2}[K_1(1+\cos\phi) + K_2(1-\cos 2\phi) + K_3(1+\cos 3\phi) + K_4(1-\cos 4\phi)]$$

**Simple harmonic:**
$$E_{dihedral} = k(1 + d \cdot \cos(n\phi))$$

Where:
- $\phi$ = dihedral angle
- $K_i$ or $k$ = force constants
- $n$ = multiplicity
- $d$ = phase shift (+1 or -1)

In [27]:
# Define dihedral style (opls for OPLS-style multi-term)
dihedral_style = ff.def_style(DihedralStyle, "opls")

# Use the style's def_type() method to create dihedral types for C-C-C-C rotation
ct_ct_ct_ct = dihedral_style.def_type(
    ct, ct, ct, ct,
    K1=1.3, K2=-0.05, K3=0.2, K4=0.0
)

print(f"Defined {len(ff.get_types(DihedralType))} dihedral types:")
for dtype in ff.get_types(DihedralType):
    K1 = dtype["K1"]
    K2 = dtype["K2"]
    K3 = dtype["K3"]
    print(f"  {dtype.name}: K1={K1:.2f}, K2={K2:.2f}, K3={K3:.2f}")

Defined 1 dihedral types:
  CT-CT-CT-CT: K1=1.30, K2=-0.05, K3=0.20


## 6. Summary of Force Field

Let's inspect what we've created:

In [28]:
print(f"Force Field: {ff.name}")
print(f"=" * 50)
print(f"Atom types:     {len(ff.get_types(AtomType))}")
print(f"Bond types:     {len(ff.get_types(BondType))}")
print(f"Angle types:    {len(ff.get_types(AngleType))}")
print(f"Dihedral types: {len(ff.get_types(DihedralType))}")

# Show all styles
from molpy.core.forcefield import Style
all_styles = ff.styles.bucket(Style)
print(f"\nStyles defined: {[s.name for s in all_styles]}")

Force Field: MyCustomFF
Atom types:     2
Bond types:     2
Angle types:    3
Dihedral types: 1

Styles defined: ['full', 'harmonic', 'harmonic', 'opls']


## 7. Merging Force Fields

You can combine parameters from multiple force fields. This is useful when:
- Using different FFs for different molecules (e.g., protein + small molecule)
- Extending an existing FF with custom parameters
- Combining water models with polymer force fields

## 6.5. Extended Force Field Types

Beyond the basic bonded interactions, MolPy supports additional force field types:

**Non-bonded interactions:**
- **`PairType`**: Non-bonded pair interactions (Lennard-Jones, Coulomb)
  - Can be defined for single atom type (self-interaction) or pairs
  - Order-independent matching

**Improper dihedrals:**
- **`ImproperType`**: Out-of-plane distortions (4-atom)
  - Maintains planarity (e.g., aromatic rings, peptide bonds)
  - Different from proper dihedrals in topology definition

Let's add these to our force field:

In [29]:
from molpy.core.forcefield import PairStyle, PairType, ImproperStyle, ImproperType

# 1. Define Pair Interactions (LJ parameters)
# Note: Use 'lj126/cut' to match the registered potential name
pair_style = ff.def_style(PairStyle, "lj126/cut")

# Use def_type() to define self-interactions
ct_pair = pair_style.def_type(ct, epsilon=0.066, sigma=3.50)
hc_pair = pair_style.def_type(hc, epsilon=0.030, sigma=2.50)

# Define cross-interaction (between different types)
ct_hc_pair = pair_style.def_type(ct, hc, epsilon=0.0443, sigma=2.95)  # Geometric mean

print(f"Defined {len(ff.get_types(PairType))} pair types:")
for ptype in ff.get_types(PairType):
    epsilon = ptype["epsilon"]
    sigma = ptype["sigma"]
    print(f"  {ptype.name}: ε={epsilon:.4f} kcal/mol, σ={sigma:.2f} Å")

# 2. Define Improper Dihedrals (for planarity)
improper_style = ff.def_style(ImproperStyle, "cvff")

# Use def_type() to define improper dihedral
# Example: C-C-C-H improper (keeps carbon planar)
# Note: central atom convention varies by force field
ct_ct_ct_hc_imp = improper_style.def_type(
    ct, ct, ct, hc,
    k=10.5, chi0=0.0  # k in kcal/mol/rad^2, chi0 in degrees
)

print(f"Defined {len(ff.get_types(ImproperType))} improper types:")
for itype in ff.get_types(ImproperType):
    k = itype["k"]
    chi0 = itype["chi0"]
    print(f"  {itype.name}: k={k:.2f} kcal/mol/rad², χ0={chi0:.1f}°")

print(f"\nForce field now has:")
print(f"  Atom types:     {len(ff.get_types(AtomType))}")
print(f"  Bond types:     {len(ff.get_types(BondType))}")
print(f"  Angle types:    {len(ff.get_types(AngleType))}")
print(f"  Dihedral types: {len(ff.get_types(DihedralType))}")
print(f"  Pair types:     {len(ff.get_types(PairType))}")
print(f"  Improper types: {len(ff.get_types(ImproperType))}")

Defined 3 pair types:
  HC: ε=0.0300 kcal/mol, σ=2.50 Å
  CT-HC: ε=0.0443 kcal/mol, σ=2.95 Å
  CT: ε=0.0660 kcal/mol, σ=3.50 Å
Defined 1 improper types:
  CT-CT-CT-HC: k=10.50 kcal/mol/rad², χ0=0.0°

Force field now has:
  Atom types:     2
  Bond types:     2
  Angle types:    3
  Dihedral types: 1
  Pair types:     3
  Improper types: 1


In [30]:
# Create a second force field (e.g., for water)
water_ff = ForceField(name="TIP3P_Water")

# Define water atom types using def_type()
water_atom_style = water_ff.def_style(AtomStyle, "full")
ow = water_atom_style.def_type("OW", mass=15.999, charge=-0.834)  # Water oxygen
hw = water_atom_style.def_type("HW", mass=1.008, charge=0.417)    # Water hydrogen

print(f"Water atom types: {len(water_ff.get_types(AtomType))}")

# Define water bond using def_type()
water_bond_style = water_ff.def_style(BondStyle, "harmonic")
ow_hw = water_bond_style.def_type(ow, hw, k=450.0, r0=0.9572)

print(f"Water bond types: {len(water_ff.get_types(BondType))}")

# Define water angle using def_type()
water_angle_style = water_ff.def_style(AngleStyle, "harmonic")
hw_ow_hw = water_angle_style.def_type(hw, ow, hw, k=55.0, theta0=104.52)

print(f"Water angle types: {len(water_ff.get_types(AngleType))}")

# Merge water FF into main FF
ff.merge(water_ff)

print(f"\nAfter merge, main FF has:")
print(f"  Atom types:  {len(ff.get_types(AtomType))}")
print(f"  Bond types:  {len(ff.get_types(BondType))}")
print(f"  Angle types: {len(ff.get_types(AngleType))}")

Water atom types: 2
Water bond types: 1
Water angle types: 1

After merge, main FF has:
  Atom types:  4
  Bond types:  3
  Angle types: 4


## 8. Loading Standard Force Fields

MolPy includes pre-defined force fields that you can load directly:

## 7.5. Converting ForceField to Potentials

Force field parameters are static data, but **Potentials** are the actual energy functions used in simulations. MolPy can automatically convert `ForceField` → `Potentials`:

**Key distinction:**
- **ForceField**: Parameter database (types, k, r0, etc.)
- **Potential**: Executable energy functions (compute forces/energies)

This conversion happens when you export to simulation engines or use MolPy's internal energy calculators.

In [31]:
# Convert force field to potentials collection
potentials = ff.to_potentials()

print(f"Created potentials from force field:")
print(f"  Total potentials: {len(potentials)}")
print(f"\nPotentials collection: {potentials}")

# Individual styles can also create potentials
bond_style = ff.get_styles(BondStyle)[0] if ff.get_styles(BondStyle) else None
angle_style = ff.get_styles(AngleStyle)[0] if ff.get_styles(AngleStyle) else None
pair_style = ff.get_styles(PairStyle)[0] if ff.get_styles(PairStyle) else None

if bond_style:
    bond_potential = bond_style.to_potential()
    print(f"\nBond potential: {bond_potential}")
    print(f"  Type: {type(bond_potential).__name__}")
    
if angle_style:
    angle_potential = angle_style.to_potential()
    print(f"\nAngle potential: {angle_potential}")

# Note: Pair potential conversion requires proper style name
# The pair style name must match registered potential (e.g., 'lj126/cut' not 'lj/cut')
print(f"\nNote: Pair style '{pair_style.name if pair_style else 'N/A'}' conversion")
print(f"  Available pair potentials: coul/cut, lj126/cut")
print(f"  For full conversion, use registered style names")

Created potentials from force field:
  Total potentials: 3

Potentials collection: [<molpy.potential.bond.harmonic.Harmonic object at 0x77cbbb6f3a70>, <molpy.potential.angle.harmonic.Harmonic object at 0x77cbbb6f1580>, <molpy.potential.pair.lj.LJ126 object at 0x77cbbb6f3350>]

Bond potential: <molpy.potential.bond.harmonic.Harmonic object at 0x77cbbb6f27b0>
  Type: Harmonic

Angle potential: <molpy.potential.angle.harmonic.Harmonic object at 0x77cbbb6add60>

Note: Pair style 'lj126/cut' conversion
  Available pair potentials: coul/cut, lj126/cut
  For full conversion, use registered style names


**How it works:**

Each `Style` knows how to create its corresponding `Potential`:
- `BondStyle("harmonic")` → `BondPotential.Harmonic`
- `AngleStyle("harmonic")` → `AnglePotential.Harmonic`
- `DihedralStyle("opls")` → `DihedralPotential.OPLS`
- `PairStyle("lj/cut")` → `PairPotential.LJ126`

**Use cases:**
1. **Energy minimization**: Use potentials to optimize geometries
2. **Force calculations**: Compute forces on atoms
3. **Simulation setup**: Export to LAMMPS/GROMACS with correct potential forms
4. **Analysis**: Calculate energy components from trajectories

See the [Potential Guide](../user-guide/potential.ipynb) for details on using potential functions.

In [32]:
# Example: Loading OPLS-AA (if available in your installation)
# from molpy.data.forcefield import load_opls_aa
# opls_ff = load_opls_aa()
# print(f"OPLS-AA has {len(opls_ff.get_types(AtomType))} atom types")

# For now, let's demonstrate the concept
print("Standard force fields include:")
print("  - OPLS-AA: All-atom optimized potentials for liquids")
print("  - AMBER: Biomolecular force fields (ff14SB, GAFF)")
print("  - CHARMM: Chemistry at Harvard macromolecular mechanics")
print("  - TraPPE: Transferable potentials for phase equilibria")
print("\nSee documentation for loading and customizing standard FFs")

Standard force fields include:
  - OPLS-AA: All-atom optimized potentials for liquids
  - AMBER: Biomolecular force fields (ff14SB, GAFF)
  - CHARMM: Chemistry at Harvard macromolecular mechanics
  - TraPPE: Transferable potentials for phase equilibria

See documentation for loading and customizing standard FFs


## 9. Applying Force Fields to Molecules

Once you have a force field, use a `Typifier` to assign types to atoms in your molecular structure:

In [33]:
# Example workflow (conceptual - see Typifier tutorial for details)
from molpy.core.atomistic import Atomistic

# 1. Build or load a molecule
mol = Atomistic()
# ... add atoms and bonds ...

# 2. Create or load force field (we already have 'ff')

# 3. Apply types using a typifier
# from molpy.typifier import CustomTypifier
# typifier = CustomTypifier(forcefield=ff)
# typifier.typify(mol)

# 4. Export to simulation format
# from molpy.io.lammps import write_lammps_data
# write_lammps_data("system.data", mol, ff)

print("Typical workflow:")
print("  1. Build/import molecular structure")
print("  2. Create or load force field")
print("  3. Assign atom types with Typifier")
print("  4. Export to LAMMPS/GROMACS/OpenMM")

Typical workflow:
  1. Build/import molecular structure
  2. Create or load force field
  3. Assign atom types with Typifier
  4. Export to LAMMPS/GROMACS/OpenMM


## 10. Exporting Force Fields

Save your custom force field for later use or export to simulation formats:

In [34]:
# Save to file (JSON format - implementation dependent)
# ff.save("my_forcefield.json")

# Export to LAMMPS format
# from molpy.io.lammps import write_lammps_forcefield
# write_lammps_forcefield("in.forcefield", ff)

# Export to GROMACS format
# from molpy.io.gromacs import write_gromacs_forcefield
# write_gromacs_forcefield("forcefield.itp", ff)

print("Force field export options:")
print("  - JSON: Save/load custom force fields")
print("  - LAMMPS: pair_coeff, bond_coeff, angle_coeff commands")
print("  - GROMACS: .itp topology files")
print("  - OpenMM: XML force field files")
print("\nSee IO module documentation for detailed export examples")

Force field export options:
  - JSON: Save/load custom force fields
  - LAMMPS: pair_coeff, bond_coeff, angle_coeff commands
  - GROMACS: .itp topology files
  - OpenMM: XML force field files

See IO module documentation for detailed export examples


## 10.5. Extending the ForceField System

MolPy's force field system is **extensible** - you can define custom `Type` and `Style` classes for novel interactions:

**When to extend:**
- Implementing new potential forms (e.g., polarizable models, reactive FFs)
- Custom bonded interactions (e.g., cross-terms, CMAP)
- Special constraints (e.g., virtual sites, Drude oscillators)
- Coarse-grained force fields with custom bead types

**Extension pattern:**
```python
# 1. Define custom Type
class CustomType(Type):
    def __init__(self, name, param1, param2, **kwargs):
        super().__init__(name, **kwargs)
        self.param1 = param1
        self.param2 = param2

# 2. Define custom Style
class CustomStyle(Style):
    def def_type(self, name, **kwargs):
        custom_type = CustomType(name, **kwargs)
        self.types.add(custom_type)
        return custom_type
    
    def to_potential(self):
        # Optional: create corresponding Potential
        from molpy.potential import CustomPotential
        return CustomPotential(self.types)

# 3. Use in ForceField
ff = ForceField()
custom_style = ff.def_style(CustomStyle, "my_custom_style")
```

In [35]:
# Example: Define a custom 3-body angular-radial coupling term
from molpy.core.forcefield import Type, Style

class AngleRadialType(Type):
    """Custom type for angle-bond cross-term (like in Class II FFs)"""
    def __init__(self, name, itom, jtom, ktom, **kwargs):
        super().__init__(name, **kwargs)
        self.itom = itom
        self.jtom = jtom  # Central atom
        self.ktom = ktom
        
    def __repr__(self):
        return f"<AngleRadialType: {self.itom.name}-{self.jtom.name}-{self.ktom.name}>"

class AngleRadialStyle(Style):
    """Style for angle-bond cross-terms"""
    def def_type(self, itom, jtom, ktom, name="", **kwargs):
        """Define angle-radial coupling type
        
        Args:
            itom: First atom type
            jtom: Central atom type
            ktom: Third atom type
            name: Optional name (defaults to itom-jtom-ktom)
            **kwargs: Parameters (e.g., k_theta_r for coupling constant)
        """
        if not name:
            name = f"{itom.name}-{jtom.name}-{ktom.name}"
        art = AngleRadialType(name, itom, jtom, ktom, **kwargs)
        self.types.add(art)
        return art

# Add to force field using the proper API
angle_radial_style = ff.def_style(AngleRadialStyle, "angle_radial")
custom_term = angle_radial_style.def_type(ct, ct, ct, k_theta_r=5.0)

print(f"Custom type added: {custom_term}")
print(f"  Parameter k_theta_r: {custom_term['k_theta_r']}")
print(f"\nForce field styles: {list(ff.styles.bucket(Style))}")
print(f"\nThis demonstrates extensibility for:")
print("  - Class II force fields (COMPASS, PCFF)")
print("  - Coarse-grained models (Martini, SDK)")
print("  - Reactive force fields (ReaxFF-like terms)")
print("  - Polarizable models (Drude, AMOEBA)")
print("  - Machine learning potentials (custom descriptors)")

Custom type added: <AngleRadialType: CT-CT-CT>
  Parameter k_theta_r: 5.0

Force field styles: [<AtomStyle: full>, <BondStyle: harmonic>, <AngleStyle: harmonic>, <DihedralStyle: opls>, <PairStyle: lj126/cut>, <ImproperStyle: cvff>, <AngleRadialStyle: angle_radial>]

This demonstrates extensibility for:
  - Class II force fields (COMPASS, PCFF)
  - Coarse-grained models (Martini, SDK)
  - Reactive force fields (ReaxFF-like terms)
  - Polarizable models (Drude, AMOEBA)
  - Machine learning potentials (custom descriptors)


## Key Takeaways

**What we learned:**
- ✅ Force fields define interaction parameters for simulations
- ✅ Atom types specify mass, charge, and LJ parameters
- ✅ Bond, angle, and dihedral types define bonded interactions
- ✅ Pair types handle non-bonded interactions (LJ, Coulomb)
- ✅ Improper types maintain planarity constraints
- ✅ Force fields can be merged to combine different parameter sets
- ✅ **ForceField → Potentials conversion** enables energy calculations
- ✅ **Extensible architecture** supports custom interaction types

**Typical workflow:**
```
Build molecule → Define/Load FF → Typify → Convert to Potentials → Export → Simulate
```

**Best practices:**
- Use standard FFs (OPLS, AMBER) when possible for validated systems
- Create custom parameters only when necessary (novel materials, etc.)
- Always validate parameters against experimental or quantum data
- Understand the ForceField vs Potential distinction
- Extend the system for specialized force fields (coarse-grained, reactive, etc.)
- Document parameter sources and assumptions

**Architecture benefits:**
- **Separation of concerns**: Data (ForceField) vs computation (Potential)
- **Reusability**: Same FF can generate different potential forms
- **Extensibility**: Add new types without modifying core code
- **Type safety**: Strong typing ensures parameter consistency

---

## Next Steps

**To apply this force field to molecules:**
- 📖 **[Typifier Guide](../user-guide/typifier.ipynb)**: Learn how to assign types to atoms
- 📖 **[IO Guide](../user-guide/io.ipynb)**: Export typed structures to simulation formats
- 📖 **[Potential Guide](../user-guide/potential.ipynb)**: Use potentials for energy calculations

**Related tutorials:**
- 📘 **[Topology](topology.ipynb)**: Understanding molecular connectivity
- 📘 **[Builder](../user-guide/builder.ipynb)**: Constructing molecular systems

**For advanced users:**
- 🔧 Custom potential functions and force field extension
- 🔧 Parameter optimization workflows
- 🔧 Force field validation and benchmarking
- 🔧 Implementing Class II or reactive force fields